In [18]:
import os
from tardigrade import KitsuneIDS
from tardigrade.utils.metrics import *
import numpy as np

In [19]:

benign_file = "tardigrade/ids/Data/weekday"
benign_netstat = "tardigrade/ids/Data/[Normal]Google_Home_Mini.pkl"
 
# Parsing of the data.
model = KitsuneIDS(True)
num_packets = model.feature_extractor(benign_file)

parsing: tardigrade/ids/Data/weekday.pcap
Reading PCAP file via Scapy...
log_path: None


19989it [00:23, 939.11it/s]

EOF Reached



20000it [00:23, 849.95it/s]


skipped: 0
written: 20000


In [20]:
 
train_params = {
    # the pcap, pcapng, or tsv file to process.
    "path": benign_file + ".csv",
    "packet_limit": np.Inf,  # the number of packets to process,
 
    # KitNET params:
    # maximum size for any autoencoder in the ensemble layer
    "maxAE": 10,
    # the number of instances taken to learn the feature mapping (the ensemble's architecture)
    "FMgrace": np.floor(0.2 * num_packets),
    # the number of instances used to train the anomaly detector (ensemble itself)
    # FMgrace+ADgrace<=num samples in normal traffic
    "ADgrace": np.floor(0.8 * num_packets),
    # directory of kitsune
    # "model_path": kitsune_path,
    # if normalize==true then kitsune does normalization automatically
    "normalize": True
}
 
model.train_model(train_params)

Feature-Mapper: train-mode, Anomaly-Detector: off-mode


2760it [00:00, 13434.16it/s]

The Feature-Mapper found a mapping: 100 features to 18 autoencoders.
Feature-Mapper: execute-mode, Anomaly-Detector: train-mode


20000it [00:17, 1119.58it/s]


Feature-Mapper: execute-mode, Anomaly-Detector: execute-mode


0.27240345991264414

In [22]:
 
malicious_traffic_plot = "port_scan.png"
benign_traffic_plot = "benign.png"
 
 
malicious_file = "tardigrade/ids/Data/Adv_Port_Scanning_SmartTV 1"
model.feature_extractor(malicious_file)
 
# malicious_file = "tardigrade/ids/Data/port_scan_attack_only"
# kitsune_threshold = "_kitsune_threshold.csv"
 
# benign_pos , kitsune_threshold = model.test_model(out_image=benign_traffic_plot,record_scores=True)
score_array_b = model.test_model(benign_file , out_image=benign_traffic_plot,record_scores=True)
 
score_array_m = model.test_model(malicious_file , out_image=malicious_traffic_plot,record_scores=True)
rmse_array = score_array_b + score_array_m

model.get_plot(rmse_array)
model.eval_metrics(rmse_array)

parsing: tardigrade/ids/Data/Adv_Port_Scanning_SmartTV 1.pcap
Reading PCAP file via Scapy...
log_path: None


5949it [00:06, 917.32it/s]

EOF Reached



5953it [00:06, 913.21it/s]


skipped: 0
written: 5953
evaluting tardigrade/ids/Data/weekday.csv
meta None
kitsune model path  kitsune.pkl


...

score saved to _kitsune_score.csv
port scan examples over threshold: 3742


5953it [00:03, 1655.04it/s]


plot path: tardigrade/ids/Data/Adv_Port_Scanning_SmartTV 1_kitsune_rmse.png
Confusion Matrix: 
True Negatives:  0
False Positives:  0
False Negatives:  3908
True Positives:  22045
Precision:  1.0
Recall:  0.8494201055754633
F1 score:  0.9185799408308679


c:\Python310\lib\site-packages\sklearn\metrics\_ranking.py:1020: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  warnings.warn(


False Positive Rate:  [nan nan nan]
True Positive Rate:  [0.         0.84942011 1.        ]
AUC Score:  nan
ROC curve saved as roc_curve.png in the current directory.
